## Statistical Arbitrage on Nq1! Es1!

Connect this to the C++ infra ASAP

In [40]:
# !pip install Alpaca-py
# !pip install --upgrade yfinance

In [41]:
import os
import json
import pandas as pd
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
import signal
import sys

In [52]:
# Alpaca keys (Edit ../config/config.json)(MAKE SURE THIS JSON IS UNTRACKED IN GITHUB, DO NOT COMMIT KEYS)

# Load config
CONFIG_PATH = "../config/config.json"
with open(CONFIG_PATH, "r") as file:
    config = json.load(file)

ALPACA_API_KEY = config["marketAdapter"]["alpacaAdapter"]["api_key"]
ALPACA_API_SECRET = config["marketAdapter"]["alpacaAdapter"]["api_secret"]
BASE_URL = config["marketAdapter"]["alpacaAdapter"]["base_url"]

In [58]:
import requests
import pandas as pd

# API URL and Headers
url = "https://data.alpaca.markets/v2/stocks/bars?symbols=QQQ&timeframe=1min&start=2025-01-21T00%3A00%3A00Z&limit=1440&adjustment=raw&feed=sip&sort=asc"
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": ALPACA_API_KEY,  # Replace with your actual API key
    "APCA-API-SECRET-KEY": ALPACA_API_SECRET  # Replace with your actual API secret
}

# Fetch the response
response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
    
    # Extract bars for SPY
    bars = data['bars']['QQQ']
    
    # Convert to DataFrame
    df = pd.DataFrame(bars)
    
    # Convert timestamp column to datetime
    df['t'] = pd.to_datetime(df['t'])
    
    # Save to CSV
    df.to_csv("spy_bars.csv", index=False)
    print("CSV saved as spy_bars.csv")
else:
    print(f"Failed to fetch data: {response.status_code}, {response.text}")

CSV saved as spy_bars.csv


## Alpaca Clients

In [ ]:
# Initialize Alpaca Historical and Trading Clients
data_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_API_SECRET)
live_data_client = StockLiveDataClient(ALPACA_API_KEY, ALPACA_API_SECRET)  # For real-time data
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_API_SECRET, paper=True)  # Paper mode enabled

In [19]:
# Trade log
trade_log = []

def fetch_historical_data(symbol: str, start_date: str, end_date: str):
    """Fetch historical data for backtesting."""
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )
    bars = data_client.get_stock_bars(request_params)
    df = bars.df.reset_index()
    df = df[df['symbol'] == symbol]  # Filter for the requested symbol
    return df


def execute_historical_trades(data):
    """Simulate trades based on strategy signals."""
    global trade_log
    for i in range(1, len(data)):
        if data['Signal'].iloc[i] != data['Signal'].iloc[i - 1]:
            action = "BUY" if data['Signal'].iloc[i] == 1 else "SELL"
            trade_log.append({
                'Date': data['timestamp'].iloc[i],
                'Action': action,
                'Price': data['close'].iloc[i]
            })

            
def fetch_real_time_data(symbol: str):
    """Fetch real-time data for the given symbol."""
    request_params = StockLatestQuoteRequest(symbol_or_symbols=symbol)
    quote = live_data_client.get_stock_latest_quote(request_params)
    data = {
        "timestamp": datetime.now(),
        "symbol": symbol,
        "open": quote[symbol].ask_price,
        "close": quote[symbol].bid_price,
        "high": quote[symbol].ask_price + 0.5,  # Approximation
        "low": quote[symbol].bid_price - 0.5,   # Approximation
        "volume": 0                            # Volume data not always available
    }
    return pd.DataFrame([data])
            
def execute_live_trade(symbol, signal):
    """Execute live trade on Alpaca."""
    side = OrderSide.BUY if signal == 1 else OrderSide.SELL
    try:
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,  # Adjust quantity as needed
            side=side,
            time_in_force=TimeInForce.DAY
        )
        trading_client.submit_order(order)
        print(f"{side.capitalize()} order placed for {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")
        
def save_to_csv(file_name="trade_log.csv"):
    """Save trade log to CSV."""
    global trade_log
    if trade_log:
        df = pd.DataFrame(trade_log)
        df.to_csv(file_name, index=False)
        print(f"Trade log saved to {file_name}.")

def signal_handler(sig, frame):
    print("\nExecution interrupted. Saving trades...")
    save_to_csv()
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

### Strategy testing

In [15]:
def apply_strategy(data):
    """Simple Moving Average Crossover Strategy."""
    data['SMA_10'] = data['close'].rolling(window=10).mean()
    data['SMA_30'] = data['close'].rolling(window=30).mean()
    data['Signal'] = 0
    data.loc[data['SMA_10'] > data['SMA_30'], 'Signal'] = 1  # Buy
    data.loc[data['SMA_10'] <= data['SMA_30'], 'Signal'] = -1  # Sell
    return data

### Main execution

Downloads trade log to csv after kernal is interupted

#### Historical testing

In [17]:
symbol = "AAPL"
start_date = "2023-01-01"
end_date = "2023-12-31"

print("Fetching historical data...")
historical_data = fetch_historical_data(symbol, start_date, end_date)

print("Applying strategy...")
historical_data = apply_strategy(historical_data)

print("Executing historical trades...")
execute_historical_trades(historical_data)

print("Saving historical trade log...")
save_to_csv("historical_trade_log.csv")

Fetching data...
Applying strategy...
Executing trades...
Trades executed. Waiting for interruption to save...


KeyboardInterrupt: 

#### Live Testing (paper mode)

In [ ]:
"""
symbol = "AAPL"
start_date = "2023-01-01"
end_date = "2023-12-31"
print("Starting live trading...")

while True:
    try:
        live_data = fetch_historical_data(symbol, "2023-12-01", "2023-12-31")  # Replace with actual fetch
        live_data = apply_strategy(live_data)
        latest_signal = live_data['Signal'].iloc[-1]
        execute_live_trade(symbol, latest_signal)
    except Exception as e:
        print(f"Live trading error: {e}")
    time.sleep(60)  # Wait 1 minute before checking again
"""